**Hello. This time, I tried and light gbm.
I use https://www.kaggle.com/code/paulorzp/jpx-simple-overfitting-model as a baseline.
I hope you enjoy reading this notebook and leave some comments.**

# Import

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction

In [ ]:
path = "../input/jpx-tokyo-stock-exchange-prediction/"
prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv")

In [ ]:
def prep_prices(prices):
    prices.Date = pd.to_datetime(prices.Date).view(int)
    prices["Volume"].fillna(1,inplace=True)
    prices.fillna(0,inplace=True)
    return prices

In [ ]:
prices = prep_prices(prices)
feats = ["Date","SecuritiesCode","Open","High","Low","Close","Volume"]

In [ ]:
prices.head()

# Split data

In [ ]:
X= prices[feats]
y = prices["Target"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)

# Light GBM

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective': 'regression'
}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=10)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = prep_prices(prices)
    sample_prediction["Prediction"] = model.predict(prices[feats])
    sample_prediction["rate"] = sample_prediction["Prediction"]/prices["Volume"]
    sample_prediction.sort_values(by = "rate", ascending=False, inplace=True)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction.sort_values(by = "SecuritiesCode", ascending=True, inplace=True)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

**That's all. Thank you for your reading!!! I would be glad if you upvote this notebook.**